In [3]:
!conda install -c conda-forge folium=0.5.0 --yes


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
import pandas as pd
import folium

In [7]:
import requests

request_parameters = {
    "client_id": 'LO0IEISGKJEWSNREW3QXACPY1YOQKCRMJEROCYQ2B3KAD0B2',
    "client_secret": 'QP3VFCYLCPEUPBOJ40TF2V4AXFJBNBCJIU4W3UIG53U0JT2Z',
    "v": '20180605',
    "section": "coffee",
    "near": "Helsinki",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [8]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [9]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]


('city', 'Helsinki', 'Helsinki')

In [10]:
d["suggestedBounds"], d["totalResults"]


({'ne': {'lat': 60.178172775811575, 'lng': 24.952162607605953},
  'sw': {'lat': 60.160364637231424, 'lng': 24.921636447256862}},
 83)

In [11]:
d["geocode"]


{'what': '',
 'where': 'helsinki',
 'center': {'lat': 60.16952, 'lng': 24.93545},
 'displayString': 'Helsinki, Finland',
 'cc': 'FI',
 'geometry': {'bounds': {'ne': {'lat': 60.29788198177167,
    'lng': 25.2415192349548},
   'sw': {'lat': 60.12648197436881, 'lng': 24.8240229894074}}},
 'slug': 'helsinki-finland',
 'longId': '72057594038586161'}

In groups we again have several different items, where type and name are irrelevant for given task. The full path to items is data.json()["response"]["groups"][0]["items"].

Let's print couple of first items to get again better idea of the structure of the data:



In [12]:
d["groups"][0].keys()


dict_keys(['type', 'name', 'items'])

In [13]:
d["groups"][0]["type"], d["groups"][0]["name"]


('Recommended Places', 'recommended')

In [14]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '50f688d5e4b023d2f274b506',
  'name': 'Kaffecentralen',
  'location': {'address': 'Fredrikinkatu 59',
   'crossStreet': 'Kansakoulukatu',
   'lat': 60.167580051384675,
   'lng': 24.932525558737044,
   'labeledLatLngs': [{'label': 'display',
     'lat': 60.167580051384675,
     'lng': 24.932525558737044}],
   'postalCode': '00100',
   'cc': 'FI',
   'city': 'Helsinki',
   'state': 'Uusimaa',
   'country': 'Suomi',
   'formattedAddress': ['Fredrikinkatu 59 (Kansakoulukatu)',
    '00100 Helsinki',
    'Suomi']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referra

In [15]:
items[1]


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ddd2d44b0fba481fc927360',
  'name': 'Patisserie Teemu & Markus',
  'location': {'address': 'Yrjönkatu 25',
   'lat': 60.16789868904846,
   'lng': 24.938189914522386,
   'labeledLatLngs': [{'label': 'display',
     'lat': 60.16789868904846,
     'lng': 24.938189914522386}],
   'postalCode': '00100',
   'cc': 'FI',
   'city': 'Helsinki',
   'state': 'Uusimaa',
   'country': 'Suomi',
   'formattedAddress': ['Yrjönkatu 25', '00100 Helsinki', 'Suomi']},
  'categories': [{'id': '4bf58dd8d48988d16a941735',
    'name': 'Bakery',
    'pluralName': 'Bakeries',
    'shortName': 'Bakery',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-4ddd2d44b0fba481fc927360-1'}

Based on this,a simple for loop over the items is collecting some relevant data from the array, and after that DataFrame is created based on this parsed data.

In [16]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 46 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,50f688d5e4b023d2f274b506,Kaffecentralen,Coffee Shop,Fredrikinkatu 59,00100,60.167580,24.932526
1,4ddd2d44b0fba481fc927360,Patisserie Teemu & Markus,Bakery,Yrjönkatu 25,00100,60.167899,24.938190
2,5aec747112f0a9002c9b92ab,La Torrefazione,Café,Mannerheimintie 22,00100,60.170721,24.936158
3,4b4cb879f964a520c0bb26e3,The Ounce,Tea Room,Fredrikinkatu 55,00100,60.167182,24.932993
4,556f2874498e103ac120a121,Kissakahvila Helkatti,Pet Café,Fredrikinkatu 55,00100,60.167274,24.933142


Some density based estimator is giving a good tip where to start a new coffee business.There's a HeatMap plugin ready in Folium, let's use that to visualize all the existing cafes to same map

In [17]:
helsinki_center = d["geocode"]["center"]
helsinki_center

{'lat': 60.16952, 'lng': 24.93545}

In [18]:
from folium import plugins

# create map of Helsinki using latitude and longitude values
map_helsinki = folium.Map(location=[helsinki_center["lat"], helsinki_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_helsinki)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_helsinki.add_child(plugins.HeatMap(hm_data))

map_helsinki

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


RESULT

Based on these results,one good location for new cafe would be in crossroad of Aleksanterinkatu and Mikonkatu



In [19]:
lat = 60.168749
lng = 24.945747
map_helsinki = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_helsinki)
map_helsinki